# 1 Imports

In [1]:
import pandas as pd
import os
import glob

# 2 Loading & processing the data

In [2]:
#getting the data from 5 different csvs and combining them in to 1 pandas dataframe
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('../Data/5_animals/', "*.csv"))),ignore_index=True)

#We drop the Recognized=False rows as there is no point in having unrecognized drawings who will not contribute to the training phase.
df = df[df["recognized"]==True]

# Lists that are shorter than 250 are useless data, being mainly lines or a simple circle not indicating anything, so I drop those aswell.
# Lists that are longer than 3000 are useless aswell as those are alost always filled in squares wich add zero value.
df = df[(df['drawing'].str.len() > 250) & (df['drawing'].str.len() <  3000)]

#the recognized column is now uneccesary and can be dropped. The index will be reset aswell.
df = df.drop(columns=['recognized']).reset_index(drop=True)

#lets check out the dataframe in its current state
df.head()


,countrycode,drawing,key_id,timestamp,word
0,US,"[[[200, 150, 116, 83, 73, 69, 67, 83, 104, 119...",4727732971765760,2017-03-06 00:28:58.856880,bee
1,GB,"[[[125, 82, 50, 17, 3, 1, 15, 36, 62, 122, 174...",5805032622522368,2017-03-25 20:57:05.022320,bee
2,CZ,"[[[96, 108, 198, 224, 239, 254, 255, 248, 210,...",5257992352038912,2017-03-13 08:35:34.597420,bee
3,US,"[[[237, 201, 130, 79, 44, 23, 5, 0, 5, 34, 73,...",6401386805723136,2017-01-23 20:39:27.306980,bee
4,US,"[[[178, 172, 111, 62, 54, 24, 16, 1, 1, 10, 17...",4558220662145024,2017-03-23 16:00:56.044640,bee


In [3]:
#finding the amount of rows for each category
df['word'].value_counts()

dog          142700
butterfly    114349
bee          110804
elephant     104708
cat          102877
Name: word, dtype: int64

As you can see above, the dataset is not balanced yet. This will be done in the next step by keeping 10k of each category.

In [4]:
#balancing of the dataset: keep 10000 rows from each animal
df = df.groupby('word').sample(n=10000, random_state=1).reset_index(drop=True)

In [5]:
df['word'].value_counts()

bee          10000
butterfly    10000
cat          10000
dog          10000
elephant     10000
Name: word, dtype: int64

Now there are 10k images of every category in the dataframe. This dataframe combining and filtering the information of the 5 animals will now be written to a csv.

In [6]:
#shuffling the dataframe
df = df.sample(frac=1).reset_index(drop=True) 
.sample(n=10000, random_state=1).reset_index(drop=True)
# Writing the combined data to a single csv
df.to_csv('../Data/five_animals.csv', index=False)